# IBDA3311 - Proyek UAS: Blur Removal Using GAN

Nama anggota kelompok:
1. Christopher Jonathan (212100170)
2. Dilivio Cullen Lemuel Tilaar (212100576)
3. Juan Christian Chandra (212100108)
4. Jody Nordberg Imanuel (191900378)

## Import Libraries

In [ ]:
import tensorflow as tf
import os
import pathlib
from matplotlib import pyplot as plt
from IPython import display

## Hyperparameters

In [ ]:
# Tentukan path untuk data latih
PATH = '/kaggle/input/a-curated-list-of-image-deblurring-datasets/DBlur/Gopro/train/'

# Tentukan subdirektori untuk gambar yang kabur dan tajam dalam data latih
BLUR = PATH + 'blur'
SHARP = PATH + 'sharp'

# Tentukan path untuk data uji
TEST_PATH = '/kaggle/input/a-curated-list-of-image-deblurring-datasets/DBlur/Gopro/test/'

# Tentukan subdirektori untuk gambar yang kabur dan tajam dalam data uji
TEST_BLUR = TEST_PATH + 'blur'
TEST_SHARP = TEST_PATH + 'sharp'

In [ ]:
# Ukuran buffer untuk pengacakan data latih
BUFFER_SIZE = 400

# Ukuran batch yang digunakan dalam pelatihan model
BATCH_SIZE = 1

# ukuran gambar yang diharapkan oleh model
IMG_WIDTH = 256
IMG_HEIGHT = 256
OUTPUT_CHANNELS = 3

In [ ]:
# constant in the generator loss function
LAMBDA = 100

## Image Preprocessing and Dataset Creation

In [ ]:
# Fungsi untuk memuat gambar dari path file tertentu
def load(file_path):
    # Ekstrak nama file gambar dari path file
    image_file = tf.strings.split(file_path, os.sep)[-1]
    
    # Baca konten gambar yang blur dari file
    image_b = tf.io.read_file(BLUR + '/' + image_file)
    # Decode konten gambar yang blur sebagai format PNG
    image_b = tf.io.decode_png(image_b)
    
    # Baca konten gambar yang tajam dari file
    image_s = tf.io.read_file(SHARP + '/' + image_file)
    # Decode konten gambar yang tajam sebagai format PNG
    image_s = tf.io.decode_png(image_s)

    # Ubah tipe data gambar yang blur menjadi float32
    input_image = tf.cast(image_b, tf.float32)
    # Ubah tipe data gambar yang tajam menjadi float32
    sharp_image = tf.cast(image_s, tf.float32)

    return input_image, sharp_image

In [ ]:
# Fungsi untuk memuat gambar uji dari path file tertentu
def load_test(file_path):
    # Ekstrak nama file gambar dari path file
    image_file = tf.strings.split(file_path, os.sep)[-1]
    
    # Baca konten gambar yang blur dari file
    image_b = tf.io.read_file(TEST_BLUR + '/' + image_file)
    # Decode konten gambar yang blur sebagai format PNG
    image_b = tf.io.decode_png(image_b)
    
    # Baca konten gambar yang tajam dari file
    image_s = tf.io.read_file(TEST_SHARP + '/' + image_file)
    # Decode konten gambar yang tajam sebagai format PNG
    image_s = tf.io.decode_png(image_s)
    
    # Ubah tipe data gambar yang blur menjadi float32
    input_image = tf.cast(image_b, tf.float32)
    # Ubah tipe data gambar yang tajam menjadi float32
    sharp_image = tf.cast(image_s, tf.float32)

    return input_image, sharp_image

In [ ]:
# Fungsi untuk mengubah ukuran gambar
# Fungsi ini untuk memastikan bahwa gambar-gambar yang masuk ke dalam model memiliki ukuran yang sesuai dengan kebutuhan model
def resize(input_image, real_image, height, width):
    # Mengubah ukuran gambar yang blur menggunakan metode Nearest Neighbor
    input_image = tf.image.resize(input_image, [height, width], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    
    # Mengubah ukuran gambar yang tajam menggunakan metode Nearest Neighbor
    real_image = tf.image.resize(real_image, [height, width], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    return input_image, real_image

In [ ]:
# Fungsi untuk melakukan pemangkasan acak (random crop) pada gambar
# Fungsi ini untuk memperkenalkan variasi dan mengurangi overfitting dalam pelatihan model GAN
def random_crop(input_image, real_image):
    # Menggabungkan kedua gambar menjadi satu tensor
    stacked_image = tf.stack([input_image, real_image], axis=0)
    # Melakukan pemangkasan acak pada tensor yang telah dibuat
    cropped_image = tf.image.random_crop(stacked_image, size=[2, IMG_HEIGHT, IMG_WIDTH, 3])
    # Mengembalikan gambar yang telah dipangkas acak
    return cropped_image[0], cropped_image[1]

In [ ]:
# Fungsi untuk normalisasi nilai pixel pada gambar
# Normalisasi ini membantu dalam meningkatkan stabilitas dan konvergensi model selama pelatihan, 
# serta memastikan bahwa input yang diberikan ke model memiliki skala nilai yang seragam.
def normalize(input_image, real_image):
    # Normalisasi ini bertujuan untuk mengubah rentang nilai pixel menjadi [-1, 1].
    # Normalisasi nilai pixel pada gambar yang blur
    input_image = (input_image / 127.5) - 1
    # Normalisasi nilai pixel pada gambar yang tajam
    real_image = (real_image / 127.5) - 1

    return input_image, real_image

In [ ]:
# Anotasi tf.function mengubah fungsi menjadi graf TensorFlow untuk kinerja yang lebih baik
@tf.function()

# Fungsi random_jitter ini memiliki tujuan untuk menerapkan sejumlah transformasi data secara acak pada 
# gambar input dan gambar target selama pelatihan model. 
# Transformasi ini bertujuan untuk memperkenalkan variasi dalam data pelatihan, sehingga model dapat belajar dengan lebih baik
def random_jitter(input_image, real_image):
    # Mengubah ukuran gambar ke dimensi yang lebih besar menggunakan fungsi resize
    input_image, real_image = resize(input_image, real_image, 286, 286)
    # Melakukan pemangkasan acak pada gambar yang telah diubah ukurannya
    input_image, real_image = random_crop(input_image, real_image)
    
    # Melakukan flip horizontal pada gambar dengan probabilitas 50%
    if tf.random.uniform(()) > 0.5:
        input_image = tf.image.flip_left_right(input_image)
        real_image = tf.image.flip_left_right(real_image)

    return input_image, real_image

In [ ]:
# Fungsi untuk memuat dan memproses gambar uji
def load_image_test(image_file):
    # Memuat gambar uji menggunakan fungsi load_test
    input_image, real_image = load_test(image_file)
    # Mengubah ukuran gambar uji ke dimensi yang diharapkan menggunakan fungsi resize
    input_image, real_image = resize(input_image, real_image, IMG_HEIGHT, IMG_WIDTH)
    # Normalisasi nilai pixel pada gambar uji menggunakan fungsi normalize
    input_image, real_image = normalize(input_image, real_image)

    return input_image, real_image

In [ ]:
# Fungsi untuk memuat dan memproses gambar latih
def load_image_train(image_file):
    # Memuat gambar latih menggunakan fungsi load
    input_image, real_image = load(image_file)
    # Melakukan jittering acak pada gambar latih menggunakan fungsi random_jitter
    input_image, real_image = random_jitter(input_image, real_image)
    # Normalisasi nilai pixel pada gambar latih menggunakan fungsi normalize
    input_image, real_image = normalize(input_image, real_image)
    
    return input_image, real_image

In [ ]:
# Membuat dataset dari daftar file gambar latih
train_dataset = tf.data.Dataset.list_files(str(SHARP + '/*.png'))

# Memetakan fungsi load_image_train ke setiap elemen dataset secara paralel
# num_parallel_calls=tf.data.AUTOTUNE memungkinkan TensorFlow untuk secara otomatis menentukan jumlah thread yang optimal 
# untuk memproses secara paralel, guna meningkatkan efisiensi.
train_dataset = train_dataset.map(load_image_train, num_parallel_calls=tf.data.AUTOTUNE)

# Mengacak dataset untuk meningkatkan variasi pada setiap epoch
# Hal ini berguna untuk memperkenalkan variasi pada setiap epoch pelatihan, 
# sehingga model tidak hanya melihat pola yang sama dalam setiap batch.
train_dataset = train_dataset.shuffle(BUFFER_SIZE)

# Membagi dataset menjadi batch-batch berukuran BATCH_SIZE
train_dataset = train_dataset.batch(BATCH_SIZE)

In [ ]:
# Membuat dataset dari daftar file gambar uji
test_dataset = tf.data.Dataset.list_files(str(TEST_SHARP + '/*.png'))
# Memetakan fungsi load_image_test ke setiap elemen dataset
test_dataset = test_dataset.map(load_image_test)
# Membagi dataset uji menjadi batch-batch berukuran BATCH_SIZE
test_dataset = test_dataset.batch(BATCH_SIZE)

## Generator

### Reflection Padding

In [ ]:
# Mendefinisikan kelas ReflectionPadding2D sebagai lapisan khusus dalam model Keras
# Untuk menambah nilai padding pada tepi gambar
class ReflectionPadding2D(tf.keras.layers.Layer):
    def __init__(self, padding=(1, 1), **kwargs):
        # Mengatur padding sebagai tupel dari parameter input
        self.padding = tuple(padding)
        # Menentukan spesifikasi input sebagai input 4 dimensi
        self.input_spec = [tf.keras.layers.InputSpec(ndim=4)]
        # Memanggil konstruktor kelas induk (super) dengan argumen tambahan jika ada
        super(ReflectionPadding2D, self).__init__(**kwargs)

    def compute_output_shape(self, s):
        # Menghitung dan mengembalikan bentuk output berdasarkan bentuk input
        return (s[0], s[1] + 2 * self.padding[0], s[2] + 2 * self.padding[1], s[3])

    def call(self, x, mask=None):
        # Mendapatkan nilai padding untuk lebar dan tinggi
        w_pad,h_pad = self.padding
        # Menggunakan tf.pad untuk menerapkan padding reflektif pada input x
        return tf.pad(x, [[0,0], [h_pad,h_pad], [w_pad,w_pad], [0,0] ], 'REFLECT')

### Residual block

In [ ]:
# Fungsi untuk membangun blok residual dalam jaringan ResNet
# Untuk membantu dalam mengatasi masalah vanishing gradient (kesalahan terhadap bobot parameter)
def res_net(input, filters, kernel_size, strides=(1, 1), apply_dropout=False):
    # Inisialisasi penentu acak untuk kernel konvolusi
    initializer = tf.random_normal_initializer(0., 0.02)
    
    # Penerapan Reflection Padding menggunakan lapisan ReflectionPadding2D yang telah didefinisikan sebelumnya
    x = ReflectionPadding2D(padding=(1,1))(input)
    # Konvolusi dengan menggunakan Conv2D, dilanjutkan dengan inisialisasi kernel dan langkah tertentu
    x = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, kernel_initializer=initializer, strides=strides)(x)
    # Normalisasi batch untuk meningkatkan stabilitas dan kecepatan konvergensi
    x = tf.keras.layers.BatchNormalization()(x)
    # Aktivasi ReLU untuk memperkenalkan non-linearitas
    x = tf.nn.relu(x)

    # Jika diterapkan dropout, lakukan dropout pada layer ini
    if apply_dropout:
        x = tf.keras.layers.Dropout(0.5)(x)
    
    # Penerapan Reflection Padding pada lapisan kedua
    x = ReflectionPadding2D(padding=(1,1))(x)
    # Konvolusi lagi dengan menggunakan Conv2D, dilanjutkan dengan normalisasi batch
    x = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, kernel_initializer=initializer, strides=strides)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
    # Menambahkan output dari blok residual dengan input awal
    merged = tf.keras.layers.Add()([input, x])
    
    # Mengembalikan output blok residual yang dihasilkan
    return merged

### Generator Model

In [ ]:
# Fungsi untuk membuat model Generator
def Generator():
    # Inisialisasi penentu acak untuk kernel konvolusi
    initializer = tf.random_normal_initializer(0., 0.02)
    # Mendefinisikan input dengan dimensi 256x256x3
    inputs = tf.keras.layers.Input(shape=[256, 256, 3])

    # Blok pertama: 7x7 Convolution dengan 64 filter
    # Penerapan Reflection Padding menggunakan lapisan ReflectionPadding2D yang telah didefinisikan
    x = ReflectionPadding2D(padding=(3,3))(inputs)
    # Konvolusi dengan menggunakan Conv2D, dilanjutkan dengan inisialisasi kernel
    x = tf.keras.layers.Conv2D(64, kernel_size=(7,7), padding='valid', kernel_initializer=initializer)(x)
    # Normalisasi batch untuk meningkatkan stabilitas dan kecepatan konvergensi
    x = tf.keras.layers.BatchNormalization()(x)
    # Aktivasi ReLU untuk memperkenalkan non-linearitas
    x = tf.nn.relu(x)

    # Blok kedua: 3x3 Convolution dengan 128 filter dan stride 2
    x = tf.keras.layers.Conv2D(128, kernel_size=(3,3), padding='same', strides=2, kernel_initializer=initializer)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.nn.relu(x)

    # Blok ketiga: 3x3 Convolution dengan 256 filter dan stride 2
    x = tf.keras.layers.Conv2D(256, kernel_size=(3,3), padding='same', strides=2, kernel_initializer=initializer)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.nn.relu(x)

    # Blok ResNet: 9 blok dengan 256 filter dan dropout
    for _ in range(9):
        x = res_net(x, 256, kernel_size=(3,3), apply_dropout=True)

    # Blok keempat: Upsampling dengan 128 filter
    x = tf.keras.layers.UpSampling2D(interpolation='bilinear')(x)
    x = tf.keras.layers.Conv2D(128, kernel_size=(3,3), padding='same', kernel_initializer=initializer)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.nn.relu(x)

    # Blok kelima: Upsampling dengan 64 filter
    x = tf.keras.layers.UpSampling2D(interpolation='bilinear')(x)
    x = tf.keras.layers.Conv2D(64, kernel_size=(3,3), padding='same', kernel_initializer=initializer)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.nn.relu(x)

    # Blok keenam: 7x7 Convolution dengan jumlah filter yang sama dengan OUTPUT_CHANNELS
    x = ReflectionPadding2D(padding=(3,3))(x)
    x = tf.keras.layers.Conv2D(filters=OUTPUT_CHANNELS, kernel_size=(7,7), padding='valid', kernel_initializer=initializer)(x)
    x = tf.nn.tanh(x)
    
    # Menggunakan layer Add untuk menambahkan input awal dan output dari blok keenam
    outputs = tf.keras.layers.Add()([x, inputs])
    # Menggunakan layer Lambda untuk membagi hasil penambahan dengan 2
    outputs = tf.keras.layers.Lambda(lambda z: z/2)(outputs)
    
    # Membuat model dengan menggunakan Model API dari Keras
    model = tf.keras.models.Model(inputs=inputs, outputs=outputs, name='Generator')
    
    return model

In [ ]:
# Membuat instance dari model Generator
generator = Generator()

### Loss functions

In [ ]:
# Menggunakan model pre-trained VGG16 sebagai bagian dari model lain
vgg = tf.keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(256,256,3))
# Membuat model yang menggunakan model VGG16 untuk perhitungan loss
loss_model = tf.keras.Model(inputs=vgg.input, outputs=vgg.get_layer('block3_conv3').output)
# Menetapkan agar model VGG16 tidak dapat dilatih kembali (freeze) untuk pembaruan bobot
loss_model.trainable = False

In [ ]:
# Fungsi untuk menghitung perceptual loss antara gambar asli (y_true) dan gambar yang dihasilkan (y_pred)
def perceptual_loss(y_true, y_pred):
    # Menghitung selisih antara hasil ekstraksi fitur dari model VGG16 untuk gambar asli dan gambar yang dihasilkan
    # Kemudian, mengambil nilai rata-rata dari seluruh selisih sebagai nilai loss
    return tf.reduce_mean(tf.square(loss_model(y_true) - loss_model(y_pred)))

In [ ]:
# Fungsi untuk menghitung loss generator
def generator_loss(disc_generated_output, gen_output, target):
    # Menghitung GAN loss (negative mean dari output discriminator terhadap gambar yang dihasilkan)
    gan_loss = -tf.reduce_mean(disc_generated_output)
    # Menghitung L2 (perceptual) loss menggunakan fungsi perceptual_loss yang telah didefinisikan sebelumnya
    l2_loss = perceptual_loss(target, gen_output)
    # Menjumlahkan GAN loss dan L2 loss dengan bobot LAMBDA sebagai faktor
    total_gen_loss = gan_loss + LAMBDA*l2_loss
    
    return total_gen_loss, gan_loss, l2_loss

## Discriminator

### Patch GAN

In [ ]:
# Fungsi untuk membuat lapisan downsample (pengurangan resolusi)
def downsample(filters, size, apply_batchnorm=True):
    # Inisialisasi penentu acak untuk kernel konvolusi
    initializer = tf.random_normal_initializer(0., 0.02)
    # Membuat Sequential model sebagai container untuk lapisan-lapisan
    result = tf.keras.Sequential()
    # Menambahkan lapisan konvolusi dengan jumlah filter, ukuran kernel, dan langkah (strides) yang ditentukan
    result.add(tf.keras.layers.Conv2D(filters, size, strides=2, padding='same', kernel_initializer=initializer, use_bias=False))
    
    # Jika diterapkan, menambahkan normalisasi batch
    if apply_batchnorm:
        result.add(tf.keras.layers.BatchNormalization())
    
    # Menambahkan lapisan aktivasi LeakyReLU untuk memperkenalkan non-linearitas
    result.add(tf.keras.layers.LeakyReLU())
    
    # Mengembalikan hasil model Sequential
    return result

In [ ]:
# Fungsi untuk membuat model Discriminator
def Discriminator():
    # Inisialisasi penentu acak untuk kernel konvolusi
    initializer = tf.random_normal_initializer(0., 0.02)
    
    # Input layer untuk gambar asli dan gambar target
    inp = tf.keras.layers.Input(shape=[256, 256, 3], name='input_image')
    tar = tf.keras.layers.Input(shape=[256, 256, 3], name='target_image')
    
    # Menggabungkan gambar asli dan gambar target sebagai input
    x = tf.keras.layers.concatenate([inp, tar])  # (batch_size, 256, 256, channels*2)
    
    # Lapisan downsample (pengurangan resolusi)
    down1 = downsample(64, 4, False)(x)  # (batch_size, 128, 128, 64)
    down2 = downsample(128, 4)(down1)  # (batch_size, 64, 64, 128)
    down3 = downsample(256, 4)(down2)  # (batch_size, 32, 32, 256)
    
    # Padding zero dan lapisan konvolusi untuk mendapatkan hasil dengan jumlah filter 512
    zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3)  # (batch_size, 34, 34, 256)
    conv = tf.keras.layers.Conv2D(512, 4, strides=1, kernel_initializer=initializer, use_bias=False)(zero_pad1)  # (batch_size, 31, 31, 512)
    
    # Normalisasi batch pada hasil konvolusi
    batchnorm1 = tf.keras.layers.BatchNormalization()(conv)
    
    # Aktivasi LeakyReLU untuk memperkenalkan non-linearitas
    leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)
    
    # Padding zero dan lapisan konvolusi untuk mendapatkan hasil dengan jumlah filter 1
    zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu)  # (batch_size, 33, 33, 512)
    last = tf.keras.layers.Conv2D(1, 4, strides=1, kernel_initializer=initializer)(zero_pad2)  # (batch_size, 30, 30, 1)
    
    # Mengembalikan model Discriminator
    return tf.keras.Model(inputs=[inp, tar], outputs=last)

In [ ]:
# Membuat instance dari model Discriminator
discriminator = Discriminator()

### Discriminator loss functions

In [ ]:
# Fungsi untuk menghitung Wasserstein Loss
def wasserstein_loss(y_true, y_pred):
    # Menghitung mean dari perkalian antara label sebenarnya (y_true) dan prediksi (y_pred)
    return tf.reduce_mean(y_true * y_pred)

In [ ]:
# Fungsi untuk menghitung loss Discriminator
def discriminator_loss(disc_real_output, disc_generated_output):
    # Menghitung Wasserstein Loss untuk sampel asli dengan label sebenarnya 1
    real_loss = wasserstein_loss(tf.ones_like(disc_real_output), disc_real_output)
    # Menghitung Wasserstein Loss untuk sampel yang dihasilkan dengan label palsu 0
    generated_loss = wasserstein_loss(tf.zeros_like(disc_real_output), disc_generated_output)
    # Menghitung total loss Discriminator sebagai rata-rata dari kedua komponen loss
    total_disc_loss = tf.reduce_mean(real_loss + generated_loss)
    
    # Mengembalikan total loss Discriminator
    return total_disc_loss

## Training

### Adam Optimizers

In [ ]:
# Untuk mengupdate bobot dari generator dan discriminator
# Definisi optimizer untuk model Generator.
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

# Definisi optimizer untuk model Discriminator
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

### Output of the Generated Image

In [ ]:
# Fungsi untuk menampilkan gambar-gambar dalam format yang telah diolah oleh model
def generate_images(model, test_input, tar):
    # Menghasilkan prediksi menggunakan model
    prediction = model(test_input, training=True)
    
    # Membuat plot
    plt.figure(figsize=(15, 15))
    
    # Daftar gambar yang akan ditampilkan dalam plot
    display_list = [test_input[0], tar[0], prediction[0]]
    
    # Judul untuk setiap gambar dalam plot
    title = ['Input Image', 'Ground Truth', 'Predicted Image']
    
    # Menampilkan setiap gambar dalam subplot
    for i in range(3):
        plt.subplot(1, 3, i+1)
        plt.title(title[i])
        # Menampilkan gambar dengan mengonversi nilai pixel ke rentang [0, 1]
        plt.imshow(display_list[i] * 0.5 + 0.5)
        plt.axis('off')
    
    # Menampilkan plot
    plt.show()

### Implementation of the Learning Algorithm

In [ ]:
# Anotasi tf.function digunakan untuk mengoptimalkan fungsi agar dapat dijalankan lebih efisien oleh TensorFlow
@tf.function()
def train_step(input_image, target, step):
    # Menggunakan GradientTape untuk menghitung gradien terhadap parameter-model
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # Menghasilkan output Generator menggunakan gambar input
        gen_output = generator(input_image, training=True)
        
        # Menghitung output Discriminator untuk gambar asli dan gambar yang dihasilkan
        disc_real_output = discriminator([input_image, target], training=True)
        disc_generated_output = discriminator([input_image, gen_output], training=True)
        
        # Menghitung loss Generator (GAN loss dan L1 loss)
        gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, gen_output, target)
        
        # Menghitung loss Discriminator
        disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

    # Menghitung gradien terhadap parameter-model Generator
    generator_gradients = gen_tape.gradient(gen_total_loss, generator.trainable_variables)
    
    # Menghitung gradien terhadap parameter-model Discriminator
    discriminator_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    # Mengaplikasikan gradien pada optimizer untuk melakukan pembaruan parameter-model Generator
    generator_optimizer.apply_gradients(zip(generator_gradients, generator.trainable_variables))
    
    # Mengaplikasikan gradien pada optimizer untuk melakukan pembaruan parameter-model Discriminator
    discriminator_optimizer.apply_gradients(zip(discriminator_gradients, discriminator.trainable_variables))

In [ ]:
def fit(train_ds, steps):
    # Mengambil satu contoh gambar input dan target dari dataset pelatihan
    example_input, example_target = next(iter(train_ds.take(1)))
    
    # Melakukan iterasi sebanyak langkah yang ditentukan
    for step, (input_image, target) in train_ds.repeat().take(steps).enumerate():
        # Menampilkan gambar input, target, dan hasil prediksi setiap 1000 iterasi
        if (step) % 1000 == 0:
            generate_images(generator, example_input, example_target)
            print(f"Step: {step//1000}k")
        
        # Melakukan satu langkah pelatihan menggunakan fungsi train_step
        train_step(input_image, target, step)

        # Mencetak titik (.) setiap 10 langkah sebagai indikator pelatihan sedang berlangsung
        if (step+1) % 10 == 0:
          print('.', end='')

In [ ]:
# Melatih model GAN pada dataset pelatihan sebanyak 150,000 langkah
fit(train_dataset, steps=150000)

## Testing

Checking how the model works on training data

In [ ]:
# Menghasilkan dan menampilkan gambar-gambar hasil prediksi untuk 10 contoh dari dataset pelatihan
for inp, tar in train_dataset.take(10):
    # Memanggil fungsi generate_images untuk setiap pasangan gambar input (inp) dan target (tar)
    generate_images(generator, inp, tar)

Checking how the model works on test data

In [ ]:
# Menghasilkan dan menampilkan gambar-gambar hasil prediksi untuk 10 contoh dari dataset pengujian
for inp, tar in test_dataset.take(10):
    # Memanggil fungsi generate_images untuk setiap pasangan gambar input (inp) dan target (tar)
    generate_images(generator, inp, tar)

### We calculate the SSIM and PSNR metrics in order to check the accuracy of generation on the training dataset
Perbedaan SSIM dan PSNR : untuk mengukur kualitas gambar dari prespektif yang berbeda.
 - SSIM : Berfokus pada kesamaan struktural dan dirancang untuk meniru persepsi manusia
 - PSNR : Matrix tradisional yang memperhatikan rasio signal-to-noise

In [ ]:
# Menghitung nilai Structural Similarity Index (SSIM) untuk 30 contoh dari dataset pelatihan
ssims = []

# Melakukan iterasi untuk 30 contoh dari dataset pelatihan
for inp, tar in train_dataset.take(30):
    # Menghitung SSIM antara gambar target (tar) dan hasil prediksi Generator untuk gambar input (inp)
    ssim = tf.image.ssim(tar, generator(inp), max_val=1, filter_size=11, filter_sigma=1.5, k1=0.01, k2=0.03)
    # Menambahkan nilai SSIM ke dalam daftar ssims
    ssims.append(ssim)

# Menghitung nilai rata-rata SSIM dari semua contoh    
tf.reduce_mean(ssims)

In [ ]:
# Menghitung nilai Peak Signal-to-Noise Ratio (PSNR) untuk 30 contoh dari dataset pelatihan
psnrs = []

# Melakukan iterasi untuk 30 contoh dari dataset pelatihan
for inp, tar in train_dataset.take(30):
    # Menghitung PSNR antara gambar target (tar) dan hasil prediksi Generator untuk gambar input (inp)
    psnr = tf.image.psnr(tar, generator(inp), max_val=1)
    # Menambahkan nilai PSNR ke dalam daftar psnrs
    psnrs.append(psnr)

# Menghitung nilai rata-rata PSNR dari semua contoh
tf.reduce_mean(psnrs)

We calculate the SSIM and PSNR metrics in order to check the accuracy of generation on the test dataset

In [ ]:
# Menghitung nilai Structural Similarity Index (SSIM) untuk 30 contoh dari dataset pengujian
ssims = []

# Melakukan iterasi untuk 30 contoh dari dataset pengujian
for inp, tar in test_dataset.take(30):
    # Menghitung SSIM antara gambar target (tar) dan hasil prediksi Generator untuk gambar input (inp)
    ssim = tf.image.ssim(tar, generator(inp), max_val=1, filter_size=11, filter_sigma=1.5, k1=0.01, k2=0.03)
    # Menambahkan nilai SSIM ke dalam daftar ssims
    ssims.append(ssim)

# Menghitung nilai rata-rata SSIM dari semua contoh pada dataset pengujian
tf.reduce_mean(ssims)

In [ ]:
# Menghitung nilai Peak Signal-to-Noise Ratio (PSNR) untuk 30 contoh dari dataset pengujian
psnrs = []

# Melakukan iterasi untuk 30 contoh dari dataset pengujian
for inp, tar in test_dataset.take(30):
    # Menghitung PSNR antara gambar target (tar) dan hasil prediksi Generator untuk gambar input (inp)
    psnr = tf.image.psnr(tar, generator(inp), max_val=1)
    # Menambahkan nilai PSNR ke dalam daftar psnrs
    psnrs.append(psnr)
    
# Menghitung nilai rata-rata PSNR dari semua contoh pada dataset pengujian
tf.reduce_mean(psnrs)